**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = "/content/drive/MyDrive/PA/data/preprocessed"

from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fri Dec 26 18:55:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0% 

**Define equation, estimate the initials, and fit to each waveform**

In [ ]:
from scipy.signal.windows import tukey
from scipy.signal import find_peaks, hilbert
from scipy.fft import rfftfreq, rfft
from scipy.optimize import curve_fit, least_squares
import numpy as np

# ------------------------------------------------------------
# Stable model (gamma > 0): y(t) = C + D*t + A*exp(-gamma*t)*sin(2π f t + phi)
# We fit in gamma, but return tau = 1/gamma to keep your downstream code the same.
# ------------------------------------------------------------
def damped_sine_baseline_gamma(t, A, gamma, f, phi, C, D):
    return C + D*t + A*np.exp(-gamma*t)*np.sin(2*np.pi*f*t + phi)

def _prep(y):
    return np.asarray(y, float)

def estimate_initials(t, y, dt):
    """
    Robust starting values for A, tau, f, phi, C, D.
    (Internally computes gamma0, but returns tau0 = 1/gamma0.)
    """
    y = _prep(y)
    t = np.asarray(t, float)

    # --- Baseline (C0, D0) via linear least squares on raw y ---
    X = np.vstack([np.ones_like(t), t]).T
    C0, D0 = np.linalg.lstsq(X, y, rcond=None)[0]

    # --- Detrend for oscillation estimation ---
    y_d = y - (C0 + D0*t)

    # --- Frequency via FFT peak with parabolic refinement ---
    w = tukey(len(y_d), 0.1)
    yw = y_d * w
    Y = np.abs(rfft(yw))
    freqs = rfftfreq(len(yw), dt)
    k = np.argmax(Y[1:]) + 1  # ignore DC bin
    f0 = max(freqs[k], 1.0)
    if 1 <= k < len(Y)-1:
        a, b, c = Y[k-1], Y[k], Y[k+1]
        denom = (a - 2*b + c)
        delta = 0.5*(a - c)/denom if denom != 0 else 0.0
        f0 = freqs[k] + delta*(freqs[1] - freqs[0])

    # --- Amplitude ---
    A0 = 0.7 * (np.max(y_d) - np.min(y_d))
    if not np.isfinite(A0) or A0 <= 0:
        A0 = 1.0

    # --- Decay from log-envelope of peaks -> gamma0 > 0 ---
    min_dist = max(1, int(0.3/(max(f0,1.0)*dt)))
    pk, _ = find_peaks(np.abs(y_d), distance=min_dist)
    if len(pk) >= 3:
        tk = t[pk]
        ak = np.abs(y_d[pk]) + 1e-12
        slope, _ = np.polyfit(tk, np.log(ak), 1)  # for decay, slope ~ -gamma
        gamma0 = max(-slope, 1e4)                 # clip to positive
    else:
        gamma0 = 1.0/10e-6  # gamma ~ 100 kHz (tau ≈ 10 μs) fallback

    # --- Phase guess (pick better of two) on detrended y ---
    try:
        phi_h = np.unwrap(np.angle(hilbert(y_d))) - 2*np.pi*f0*t
        phi_h = (np.mean(phi_h) + np.pi) % (2*np.pi) - np.pi
    except Exception:
        phi_h = 0.0

    idx = int(np.argmax(np.abs(y_d)))
    if idx == 0: idx = 1
    dy = y_d[idx] - y_d[idx-1]
    phi_d = np.arctan2(dy/dt, 2*np.pi*f0*y_d[idx])

    s_h = np.corrcoef(y_d, A0*np.exp(-gamma0*t)*np.sin(2*np.pi*f0*t + phi_h))[0,1]
    s_d = np.corrcoef(y_d, A0*np.exp(-gamma0*t)*np.sin(2*np.pi*f0*t + phi_d))[0,1]
    phi0 = phi_h if np.abs(s_h) >= np.abs(s_d) else phi_d

    tau0 = 1.0/gamma0  # return tau for compatibility
    return float(A0), float(tau0), float(f0), float(phi0), float(C0), float(D0)

def fit_one(t, y, dt, bounds=None, robust=False):
    """
    Fit damped sinusoid + linear baseline using stable gamma parameterization.
    Returns popt, yhat, rmse, nrmse with popt = [A, tau, f, phi, C, D]
    Set robust=True to use soft-L1 loss (better with spikes/outliers).
    """
    t = np.asarray(t, float)
    y = np.asarray(y, float)

    # Initials (returns tau0; convert to gamma0)
    A0, tau0, f0, phi0, C0, D0 = estimate_initials(t, y, dt)
    gamma0 = max(1.0/abs(tau0), 1e4)  # ensure > 0

    # Bounds (in gamma space)
    if bounds is None:
        # A, gamma, f, phi, C, D
        bounds = (
            [0.0, 1e4, 5e3, -np.pi, -1e6, -1e9],
            [1e9, 1e7, 3e5,  np.pi,  1e6,  1e9],
        )

    # Build seeds (vary f and phi; keep C,D from LS; use gamma0)
    seeds = []
    for fscale in [0.95, 1.0, 1.05]:
        for dphi in [0.0, np.pi/2, -np.pi/2]:
            seeds.append([
                max(A0, 1e-12),
                np.clip(gamma0, bounds[0][1], bounds[1][1]),
                np.clip(f0 * fscale, bounds[0][2], bounds[1][2]),
                ((phi0 + dphi + np.pi) % (2*np.pi)) - np.pi,
                np.clip(C0, bounds[0][4], bounds[1][4]),
                np.clip(D0, bounds[0][5], bounds[1][5]),
            ])

    best = None

    if robust:
        # Robust solver (soft-L1)
        def resid_fun(p):
            return y - damped_sine_baseline_gamma(t, *p)
        for p0 in seeds:
            try:
                res = least_squares(resid_fun, p0, bounds=bounds, loss='soft_l1', f_scale=1.0, max_nfev=20000)
                p = res.x
                yhat = damped_sine_baseline_gamma(t, *p)
                r = y - yhat
                rmse = np.sqrt(np.mean(r**2))
                nrmse = rmse / (np.ptp(y) + 1e-12)
                score = nrmse
                if (best is None) or (score < best[0]):
                    best = (score, p, yhat, rmse, nrmse)
            except Exception:
                continue
    else:
        # Classic LS (curve_fit)
        for p0 in seeds:
            try:
                popt_g, _ = curve_fit(damped_sine_baseline_gamma, t, y, p0=p0, bounds=bounds, maxfev=20000)
                yhat = damped_sine_baseline_gamma(t, *popt_g)
                r = y - yhat
                rmse = np.sqrt(np.mean(r**2))
                nrmse = rmse / (np.ptp(y) + 1e-12)
                score = nrmse
                if (best is None) or (score < best[0]):
                    best = (score, popt_g, yhat, rmse, nrmse)
            except Exception:
                continue

    if best is None:
        # Fall back to initials
        popt_g = np.array([A0, gamma0, f0, phi0, C0, D0], float)
        yhat = damped_sine_baseline_gamma(t, *popt_g)
        r = y - yhat
        rmse = np.sqrt(np.mean(r**2))
        nrmse = rmse / (np.ptp(y) + 1e-12)
    else:
        _, popt_g, yhat, rmse, nrmse = best

    # Convert back to tau for output compatibility
    A, gamma, f, phi, C, D = popt_g
    tau = 1.0 / gamma
    popt = np.array([A, tau, f, phi, C, D], float)

    return popt, yhat, rmse, nrmse


# ---------- Windowed fitting utilities ----------

def _coarse_freq(y, dt):
    from scipy.fft import rfft, rfftfreq
    Y = np.abs(rfft(y - np.mean(y)))
    F = rfftfreq(len(y), dt)
    k = np.argmax(Y[1:]) + 1
    f0 = max(F[k], 1.0)
    if 1 <= k < len(Y)-1:
        a, b, c = Y[k-1], Y[k], Y[k+1]
        denom = (a - 2*b + c)
        delta = 0.5*(a - c)/denom if denom != 0 else 0.0
        f0 = F[k] + delta*(F[1]-F[0])
    return f0

def _make_local_bounds(prev, ratios, hard_bounds=None):
    """
    Build bounds around previous params (A, tau, f, phi, C, D) but RETURN them
    in (A, gamma, f, phi, C, D) because fit_one() fits gamma.
    ratios: multiplicative for A,tau,f; additive for phi,C,D.
    """
    A, tau, f, phi, C, D = prev
    tau = float(max(tau, 1e-9))

    # propose tau-bounds around the previous value
    A_lo, A_hi   = A*(1-ratios['A']),   A*(1+ratios['A'])
    tau_lo, tau_hi = tau*(1-ratios['tau']), tau*(1+ratios['tau'])
    f_lo, f_hi   = f*(1-ratios['f']),   f*(1+ratios['f'])
    phi_lo,phi_hi= phi - ratios['phi'], phi + ratios['phi']
    C_lo,  C_hi  = C - ratios['C'],     C + ratios['C']
    D_lo,  D_hi  = D - ratios['D'],     D + ratios['D']

    # convert tau-bounds -> gamma-bounds (monotone invert, swap lo/hi)
    tau_lo = max(tau_lo, 1e-12)
    tau_hi = max(tau_hi, 1.01*tau_lo)
    gamma_lo = 1.0 / tau_hi   # smaller gamma = longer tau
    gamma_hi = 1.0 / tau_lo   # larger gamma = shorter tau

    lo = np.array([A_lo, gamma_lo, f_lo, phi_lo, C_lo, D_lo], float)
    hi = np.array([A_hi, gamma_hi, f_hi, phi_hi, C_hi, D_hi], float)

    if hard_bounds is not None:
        # hard_bounds must also be in (A, gamma, f, phi, C, D)
        lo = np.maximum(lo, hard_bounds[0])
        hi = np.minimum(hi, hard_bounds[1])

    return (lo, hi)


def fit_sliding_windows(t, y, dt, cycles=6, overlap=0.5,
                        robust=True,
                        local_ratio=None,
                        use_weight='tukey'):
    """
    Windowed fitting with overlap-add.
    Returns:
        yhat_oa : overlap-add reconstruction over full t
        params  : array (nwin, 6) of [A, tau, f, phi, C, D] per window
        nrmse   : array (nwin,) window nrmse
        idxs    : list of slice objects for each window on original t
        meta    : dict with L, H, f0
    """
    y = np.asarray(y, float)
    t = np.asarray(t, float)

    # coarse frequency to set window length ~ cycles/f0
    f0 = _coarse_freq(y, dt)
    L = int(max(64, np.round((cycles / max(f0, 1.0)) / dt)))  # samples per window
    H = int(max(1, np.round(L * (1 - overlap))))              # hop
    if L > len(y):  # degenerate case
        L = len(y)
        H = max(1, L // 2)

    # --- analysis/synthesis windows ---
    # Interior frames: tapered; first/last: rectangular to avoid edge collapse
    if use_weight == 'tukey':
        w_core = tukey(L, 0.25)
    else:
        w_core = np.hanning(L)
    w_edge = np.ones(L)

    yhat_oa = np.zeros_like(y, float)
    wsum    = np.zeros_like(y, float)

    params_list = []
    nrmse_list  = []
    idxs        = []

    prev = None
    base_bounds = (
        [0.0, 1e4, 5e3, -np.pi, -1e9, -1e12],   # A, gamma, f, phi, C, D
        [1e9, 1e7, 3e5,  np.pi,  1e9,  1e12],
    )

    # default "slow drift" local constraints
    if local_ratio is None:
        local_ratio = dict(A=0.5, tau=0.5, f=0.2, phi=np.pi/3, C=5e4, D=1e10)

    for wi, start in enumerate(range(0, len(y) - L + 1, H)):
        sl = slice(start, start + L)
        idxs.append(sl)
        t_win = t[sl]; y_win = y[sl]

        # build local bounds around previous params if available
        if prev is not None and np.all(np.isfinite(prev)):
            bounds_local = _make_local_bounds(prev, local_ratio, base_bounds)
        else:
            bounds_local = base_bounds

        popt, yhat_win, rmse, nrm = fit_one(t_win, y_win, dt,
                                            bounds=bounds_local,
                                            robust=robust)

        params_list.append(popt)
        nrmse_list.append(nrm)

        # --- edge-safe overlap-add ---
        is_last = (start + L >= len(y))
        w_use = w_edge if (wi == 0 or is_last) else w_core
        yhat_oa[sl] += w_use * yhat_win
        wsum[sl]    += w_use

        prev = popt

    # avoid division by tiny numbers at very edges
    wsum_safe = np.where(wsum > 1e-9, wsum, 1.0)
    yhat_oa /= wsum_safe

    params_arr = np.vstack(params_list)
    nrmse_arr  = np.array(nrmse_list)

    meta = dict(L=L, H=H, f0=f0, cycles=cycles, overlap=overlap)
    return yhat_oa, params_arr, nrmse_arr, idxs, meta


def summarize_windows(params_arr, nrmse_arr, mode='nrmse'):
    """
    Weighted summary of window parameters.
    Returns a 6-vector [A, tau, f, phi, C, D] (phi left as circular mean).
    """
    A, tau, f, phi, C, D = params_arr.T
    gamma = 1/np.clip(tau, 1e-12, None)

    if mode == 'nrmse':
        w = 1/np.clip(nrmse_arr, 1e-6, None)**2
    elif mode == 'amplitude':
        w = np.clip(A, 1e-12, None)**2
    else:
        w = np.ones_like(A)

    def wmean(x): return float(np.sum(w*x)/np.sum(w))
    Abar = wmean(A)
    gbar = wmean(gamma); taubar = 1.0/max(gbar, 1e-12)
    fbar = wmean(f)
    Cbar = wmean(C); Dbar = wmean(D)

    # circular mean for phase (optional; often refit is better)
    sinp = wmean(np.sin(phi)); cosp = wmean(np.cos(phi))
    phibar = float(np.arctan2(sinp, cosp))
    return np.array([Abar, taubar, fbar, phibar, Cbar, Dbar], float)





**Inverting signal**

In [ ]:
# =========================
# Load raw data + invert Bank A & C (before fitting)
# =========================
import os, json
import numpy as np


# --- CONFIG ---
replicate = 2
version   = "lowering"

base_pre  = "/content/drive/MyDrive/data/preprocessed"
base_ring = "/content/drive/MyDrive/data/ringdown"

tensor_path = os.path.join(base_ring, f"tensor_replicate_{replicate}_{version}_ringdown.npy")
press_path  = os.path.join(base_pre,  f"pressures_replicate_{replicate}_{version}.json")
nozz_path   = os.path.join(base_pre,  f"nozzles_replicate_{replicate}_{version}.json")

# --- LOAD ---
if not os.path.exists(tensor_path):
    raise FileNotFoundError(f"Missing tensor: {tensor_path}")
if not os.path.exists(press_path):
    raise FileNotFoundError(f"Missing pressures json: {press_path}")
if not os.path.exists(nozz_path):
    raise FileNotFoundError(f"Missing nozzles json: {nozz_path}")

tensor = np.load(tensor_path, allow_pickle=True)  # (n_nozzles, n_samples, n_pressures)
with open(press_path, "r") as f:
    pressures = np.array(json.load(f), dtype=float)
with open(nozz_path, "r") as f:
    nozzles = json.load(f)

n_nozzles, n_samples, n_pressures = tensor.shape

# --- TIMEBASE (matches your old notebook) ---
fs = 4.125e6
dt = 1.0 / fs
t  = np.arange(n_samples) * dt

print(f" Loaded tensor: {tensor.shape} (nozzles × samples × pressures)")
print(f" Loaded pressures: {len(pressures)} | nozzles: {len(nozzles)}")
print(f" Timebase: fs={fs:.3e} Hz, dt={dt:.3e} s, t.shape={t.shape}")

# --- BANK MASK (A/C inverted, B/D not) ---
nozz_arr = np.array([str(n) for n in nozzles], dtype=object)
banks    = np.array([s[0].upper() if len(s) else "?" for s in nozz_arr], dtype=object)

mask_AC_nozzle = (banks == "A") | (banks == "C")                 # (n_nozzles,)
inverted_mask  = np.repeat(mask_AC_nozzle[:, None], n_pressures, axis=1)  # (n_nozzles, n_pressures)

print(f" Will invert: {mask_AC_nozzle.sum()} / {n_nozzles} nozzles (banks A or C)")

# --- INVERT (reflection around mid-value, not y -> -y) ---
# Keep a raw copy in case you want to compare later
tensor_raw = tensor.copy()

# per-(nozzle, pressure) min/max over the sample axis
y_min = tensor.min(axis=1)          # (n_nozzles, n_pressures)
y_max = tensor.max(axis=1)          # (n_nozzles, n_pressures)
y_mid = 0.5 * (y_min + y_max)       # (n_nozzles, n_pressures)

tensor_inv = 2.0 * y_mid[:, None, :] - tensor  # broadcast to (n_nozzles, n_samples, n_pressures)
tensor[mask_AC_nozzle, :, :] = tensor_inv[mask_AC_nozzle, :, :]

print(" Applied bank A/C inversion to `tensor` (and left B/D unchanged).")
print("   - `tensor_raw` contains the original signals.")
print("   - `inverted_mask` is True for (A/C, all pressures) and can be saved with results if you want.")



 Loaded tensor: (1280, 89, 95) (nozzles × samples × pressures)
 Loaded pressures: 95 | nozzles: 1280
 Timebase: fs=4.125e+06 Hz, dt=2.424e-07 s, t.shape=(89,)
 Will invert: 640 / 1280 nozzles (banks A or C)
 Applied bank A/C inversion to `tensor` (and left B/D unchanged).
   - `tensor_raw` contains the original signals.
   - `inverted_mask` is True for (A/C, all pressures) and can be saved with results if you want.


**New fitting loop**

In [ ]:
# =========================
# OA-seed fitting loop (Option A)
# Uses existing in-memory: tensor, pressures, nozzles (already bank-inverted A/C)
# IMPORTANT: does NOT use maybe_reflect_signal / phase mask
# =========================
import os, re, datetime
import numpy as np
import multiprocessing
from joblib import Parallel, delayed

# ------------ CONFIG ------------
replicate = 2
version   = "lowering"

# Output directory for partial fits
OUT_DIR = f"/content/drive/MyDrive/data/ringdown/oa_fit_results_rep_{replicate}_{version}_phaseflipped_negphi"
os.makedirs(OUT_DIR, exist_ok=True)

# ------------ REQUIRE preloaded variables (from your inversion cell) ------------
required = ["tensor", "pressures", "nozzles"]
missing = [v for v in required if v not in globals()]
if missing:
    raise NameError(
        f"Missing {missing}. Run your 'Load raw data + invert Bank A & C' cell FIRST, "
        "then run this fitting cell."
    )

# Ensure shapes / timebase
n_nozzles, n_samples, n_pressures = tensor.shape

# Use existing t/dt if present; otherwise build them
if "dt" not in globals() or "t" not in globals():
    fs = 4.125e6
    dt = 1.0 / fs
    t  = np.arange(n_samples) * dt
else:
    # sanity check: if t length mismatches, rebuild to match tensor
    if len(t) != n_samples:
        fs = 4.125e6
        dt = 1.0 / fs
        t  = np.arange(n_samples) * dt

print(f" Using in-memory tensor: {tensor.shape} (nozzles × samples × pressures)")
print(f" Using in-memory pressures: {len(pressures)} | nozzles: {len(nozzles)}")
print(f" Partials folder: {OUT_DIR}")

# ------------ Windowed fit hyperparams (fallbacks) ------------
try: WIN_CYCLES
except NameError: WIN_CYCLES = 6.0
try: WIN_OVERLAP
except NameError: WIN_OVERLAP = 0.50

# ------------ Resume logic ------------
print(f"Resuming / writing partials in folder: {OUT_DIR}")

pat = re.compile(r"^params6_partial_(\d+)_(\d+)\.npy$")
done_ranges = []
for fname in os.listdir(OUT_DIR):
    m = pat.match(fname)
    if m:
        s, e = int(m.group(1)), int(m.group(2))
        done_ranges.append((s, e))
done_ranges.sort()

if done_ranges:
    max_done = max(e for (_, e) in done_ranges)
    RESUME_FROM = max_done + 1
else:
    RESUME_FROM = 0

print("Already completed ranges:", done_ranges[:5], "..." if len(done_ranges) > 5 else "")
print(f"  Will resume from nozzle index: {RESUME_FROM}")

done_batch_set = set(done_ranges)

# ------------ Fitting function for one nozzle ------------
# NOTE: We intentionally do NOT call maybe_reflect_signal().
#       tensor is assumed already bank-inverted (A/C) from your previous cell.
def fit_nozzle(local_idx: int):
    local_params6 = np.full((n_pressures, 6), np.nan, dtype=float)
    local_Q       = np.full(n_pressures,     np.nan, dtype=float)
    local_rmse    = np.full(n_pressures,     np.nan, dtype=float)
    local_nrmse   = np.full(n_pressures,     np.nan, dtype=float)

    for k in range(n_pressures):
        y = tensor[local_idx, :, k]  # <-- already corrected upstream (A/C inversion)

        try:
            yhat_k, p_win_k, nrmse_win_k, _, _ = fit_sliding_windows(
                t, y, dt, cycles=WIN_CYCLES, overlap=WIN_OVERLAP, robust=True
            )
            p_seed_k = summarize_windows(p_win_k, nrmse_win_k, mode="nrmse")  # [A, tau, f, phi, C, D]
            A, tau, f, phi, C, D = p_seed_k

            rmse_k  = np.sqrt(np.mean((y - yhat_k) ** 2))
            nrmse_k = rmse_k / (np.ptp(y) + 1e-12)
            Q_k     = np.pi * f * tau if (np.isfinite(f) and np.isfinite(tau) and tau > 0) else np.nan

            local_params6[k, :] = p_seed_k
            local_Q[k]          = Q_k
            local_rmse[k]       = rmse_k
            local_nrmse[k]      = nrmse_k

        except Exception as e:
            print(f" OA fit failed for nozzle {local_idx}, pressure {k}: {e}")

    return local_params6, local_Q, local_rmse, local_nrmse

# ------------ Batch loop with Parallel ------------
n_jobs     = -1
batch_size = 50
print(f" Fitting started at {datetime.datetime.now().strftime('%H:%M:%S')} using {n_jobs} cores...")

for batch_start in range(RESUME_FROM, n_nozzles, batch_size):
    batch_end    = min(batch_start + batch_size, n_nozzles)
    global_start = batch_start
    global_end   = batch_end - 1

    if (global_start, global_end) in done_batch_set:
        print(f"  Skipping existing batch {global_start}-{global_end}")
        continue

    print(f"\n Processing nozzles {global_start}–{global_end}...")

    results = Parallel(n_jobs=n_jobs, verbose=5)(
        delayed(fit_nozzle)(i) for i in range(batch_start, batch_end)
    )

    params6_batch = np.array([r[0] for r in results])   # (B, n_pressures, 6)
    Q_batch       = np.array([r[1] for r in results])   # (B, n_pressures)
    rmse_batch    = np.array([r[2] for r in results])   # (B, n_pressures)
    nrmse_batch   = np.array([r[3] for r in results])   # (B, n_pressures)

    np.save(os.path.join(OUT_DIR, f"params6_partial_{global_start}_{global_end}.npy"), params6_batch)
    np.save(os.path.join(OUT_DIR, f"Q_partial_{global_start}_{global_end}.npy"),       Q_batch)
    np.save(os.path.join(OUT_DIR, f"rmse_partial_{global_start}_{global_end}.npy"),    rmse_batch)
    np.save(os.path.join(OUT_DIR, f"nrmse_partial_{global_start}_{global_end}.npy"),   nrmse_batch)

    print(f"💾 Saved partial batch {global_start}-{global_end}")

print(" Fitting completed.")
print(f" Finished at {datetime.datetime.now().strftime('%H:%M:%S')}")



 Using in-memory tensor: (1280, 89, 95) (nozzles × samples × pressures)
 Using in-memory pressures: 95 | nozzles: 1280
 Partials folder: /content/drive/MyDrive/data/ringdown/oa_fit_results_rep_2_lowering_phaseflipped_negphi
Resuming / writing partials in folder: /content/drive/MyDrive/data/ringdown/oa_fit_results_rep_2_lowering_phaseflipped_negphi
Already completed ranges: [(0, 49), (50, 99), (100, 149), (150, 199), (200, 249)] ...
  Will resume from nozzle index: 800
 Fitting started at 18:41:25 using -1 cores...

 Processing nozzles 800–849...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KeyboardInterrupt: 

**New merge cell**

In [ ]:
# ==== Merge PHASE-CORRECTED partials and save merged NPZ ====
import os, re, json
import numpy as np



# --------- DESTINATION (fixed) ----------
OUT_DIR = "/content/drive/MyDrive/data/fit"  # keep same main fit folder
os.makedirs(OUT_DIR, exist_ok=True)

replicate = 2
version = "lowering"
base_pre  = "/content/drive/MyDrive/data/preprocessed"
base_ring = "/content/drive/MyDrive/data/ringdown"

# --------- SOURCE (where NEW partial files live) ----------
SRC_DIR = f"/content/drive/MyDrive/data/ringdown/oa_fit_results_rep_{replicate}_{version}_phaseflipped_negphi"  # <-- must match OUT_DIR

# --- load original tensor / meta to know shapes ---
tensor_path = os.path.join(base_ring, f"tensor_replicate_{replicate}_{version}_ringdown.npy")
press_path  = os.path.join(base_pre,  f"pressures_replicate_{replicate}_{version}.json")
nozz_path   = os.path.join(base_pre,  f"nozzles_replicate_{replicate}_{version}.json")

tensor = np.load(tensor_path, allow_pickle=True)
with open(press_path, "r") as f: pressures = np.array(json.load(f), dtype=float)
with open(nozz_path,  "r") as f: nozzles   = json.load(f)

n_nozzles, n_samples, n_pressures = tensor.shape
print(f" Tensor: {tensor.shape}  (nozzles × samples × pressures)")
print(f" Reading PHASE-CORRECTED partials from: {SRC_DIR}")
print(f" Will save merged file into: {OUT_DIR}")

# --- discover partials ---
pat = re.compile(r"^(params6|Q|rmse|nrmse)_partial_(\d+)_(\d+)\.npy$")
files = {}
for fname in os.listdir(SRC_DIR):
    m = pat.match(fname)
    if not m:
        continue
    kind, s, e = m.group(1), int(m.group(2)), int(m.group(3))
    files.setdefault((s, e), {})[kind] = os.path.join(SRC_DIR, fname)

if not files:
    raise FileNotFoundError("No phase-corrected partial files found in SRC_DIR.")

batches = sorted(files.items(), key=lambda kv: (kv[0][0], kv[0][1]))
labels  = ", ".join(f"{s}-{e}" for (s, e), _ in batches[:6])
more    = "..." if len(batches) > 6 else ""
print(f"🧩 Found {len(batches)} batch ranges: [{labels}] {more}")

# --- allocate merged arrays ---
params6_all = np.full((n_nozzles, n_pressures, 6), np.nan, dtype=float)
Q_all       = np.full((n_nozzles, n_pressures),    np.nan, dtype=float)
rmse_all    = np.full((n_nozzles, n_pressures),    np.nan, dtype=float)
nrmse_all   = np.full((n_nozzles, n_pressures),    np.nan, dtype=float)

filled   = np.zeros(n_nozzles, dtype=bool)
overlaps = []

# --- merge loop ---
for (s, e), group in batches:
    if "params6" not in group:
        print(f"⏭️  Skipping {s}-{e}: missing params6 file.")
        continue

    p6 = np.load(group["params6"])  # (B, n_pressures, 6)
    B  = p6.shape[0]
    if p6.ndim != 3 or p6.shape[1] != n_pressures or p6.shape[2] != 6:
        raise ValueError(f"Shape mismatch in {group['params6']}: {p6.shape} != (B,{n_pressures},6)")
    if B != (e - s + 1):
        raise ValueError(f"Range {s}-{e} implies B={e-s+1}, got {B}")

    Q  = np.load(group["Q"])     if "Q"     in group else None
    R  = np.load(group["rmse"])  if "rmse"  in group else None
    NR = np.load(group["nrmse"]) if "nrmse" in group else None

    if Q  is not None and Q.shape  != (B, n_pressures):   raise ValueError(f"Q shape mismatch: {Q.shape}")
    if R  is not None and R.shape  != (B, n_pressures):   raise ValueError(f"rmse shape mismatch: {R.shape}")
    if NR is not None and NR.shape != (B, n_pressures):   raise ValueError(f"nrmse shape mismatch: {NR.shape}")

    sl = slice(s, e+1)
    if np.any(filled[sl]):
        overlaps.append((s, e, np.where(filled[sl])[0] + s))

    params6_all[sl, :, :] = p6
    if Q  is not None: Q_all[sl, :]     = Q
    if R  is not None: rmse_all[sl, :]  = R
    if NR is not None: nrmse_all[sl, :] = NR
    filled[sl] = True

covered = np.where(filled)[0]
missing = np.where(~filled)[0]
print(f"\n📊 Coverage: {covered.size}/{n_nozzles} nozzles ({covered.size/n_nozzles:.1%}) merged.")
if overlaps:
    overlap_labels = ", ".join(f"{s}-{e}" for (s, e, _) in overlaps[:3])
    more = "..." if len(overlaps) > 3 else ""
    print(f"ℹ️ Overlaps detected; later files overwrote earlier for ranges: [{overlap_labels}] {more}")
if missing.size:
    miss_path = os.path.join(OUT_DIR, "missing_nozzle_indices_phaseflipped.txt")
    np.savetxt(miss_path, missing, fmt="%d")
    print(f"⚠️ Missing {missing.size} nozzle(s). Saved list to: {miss_path}")

# --- load neg_phase_mask from SRC_DIR ---
neg_mask_path = os.path.join(SRC_DIR, "neg_phase_mask.npy")
if os.path.exists(neg_mask_path):
    neg_phase_mask = np.load(neg_mask_path)
    assert neg_phase_mask.shape == (n_nozzles, n_pressures), \
        f"neg_phase_mask shape {neg_phase_mask.shape} != {(n_nozzles, n_pressures)}"
    print(" Loaded neg_phase_mask from phase-corrected run.")
else:
    neg_phase_mask = np.zeros((n_nozzles, n_pressures), dtype=bool)
    print(" neg_phase_mask.npy not found in SRC_DIR, using all-False mask.")

# --- save merged PHASE-CORRECTED npz ---
merged_path = os.path.join(
    OUT_DIR,
    f"merged_rep_{replicate}_{version}_phaseflipped_params6_Q_rmse_nrmse.npz"
)

np.savez_compressed(
    merged_path,
    params6_all=params6_all,
    Q_all=Q_all,
    rmse_all=rmse_all,
    nrmse_all=nrmse_all,
    pressures=pressures,
    nozzles=np.array(nozzles, dtype=object),
    neg_phase_mask=neg_phase_mask,
)

print(f"\n Saved PHASE-CORRECTED merged file: {merged_path}")

# Quick sanity ping
if covered.size:
    i, k = int(covered[0]), min(10, n_pressures-1)
    ok = np.all(np.isfinite(params6_all[i, k]))
    print(f"Sanity check — nozzle {i}, pressure {k}: params finite = {ok}")

 Tensor: (1280, 89, 95)  (nozzles × samples × pressures)
 Reading PHASE-CORRECTED partials from: /content/drive/MyDrive/data/ringdown/oa_fit_results_rep_2_lowering_phaseflipped_negphi
 Will save merged file into: /content/drive/MyDrive/data/fit
🧩 Found 16 batch ranges: [0-49, 50-99, 100-149, 150-199, 200-249, 250-299] ...

📊 Coverage: 800/1280 nozzles (62.5%) merged.
⚠️ Missing 480 nozzle(s). Saved list to: /content/drive/MyDrive/data/fit/missing_nozzle_indices_phaseflipped.txt
 neg_phase_mask.npy not found in SRC_DIR, using all-False mask.

 Saved PHASE-CORRECTED merged file: /content/drive/MyDrive/data/fit/merged_rep_2_lowering_phaseflipped_params6_Q_rmse_nrmse.npz
Sanity check — nozzle 0, pressure 10: params finite = True


****Build label_all[i,j]****

In [ ]:
import os, re, glob, json
import numpy as np
import pandas as pd

# ---- CONFIG ----
replicate = 2
version   = "lowering"

base_pre  = "/content/drive/MyDrive/data/preprocessed"
fit_dir   = "/content/drive/MyDrive/data/fit"

LABELS_DIR = {
  "lowering":   "/content/drive/MyDrive/data/labels/2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps_ISPM_PH4",
  "increasing": "/content/drive/MyDrive/data/labels/2025-05-27-03-22-58-PM_Gradually_Increasing_Pressure_0dot4mBar_Steps_ISPM_PH4",
}[version]

# ---- LOAD PRESSURES + NOZZLES ----
press_path = os.path.join(base_pre, f"pressures_replicate_{replicate}_{version}.json")
nozz_path  = os.path.join(base_pre, f"nozzles_replicate_{replicate}_{version}.json")

with open(press_path, "r") as f:
    pressures = np.array(json.load(f), dtype=float)
with open(nozz_path, "r") as f:
    nozzles = np.array(json.load(f), dtype=object)

n_nozzles   = len(nozzles)
n_pressures = len(pressures)

# Allowed pressure range comes from the dataset itself
P_LOW  = float(np.min(pressures))
P_HIGH = float(np.max(pressures))

# Tolerance: half a step (robust). If step is weird, fallback to 0.25 mBar
steps = np.diff(np.sort(pressures))
step_med = float(np.median(np.abs(steps))) if steps.size else 0.4
TOL = max(0.25, 0.51 * step_med)

print("n_nozzles:", n_nozzles, "n_pressures:", n_pressures)
print(f"Pressure grid range: [{P_LOW:.3f}, {P_HIGH:.3f}] mBar | step≈{step_med:.3f} | tol={TOL:.3f}")
print("LABELS_DIR:", LABELS_DIR)

# ---- canonical nozzle helper ----
def canonical_nozzle(name: str) -> str:
    s = str(name).strip()
    m = re.search(r'([ABCD])[^0-9]*([0-9]+)', s)
    if not m:
        return s
    bank = m.group(1)
    num  = int(m.group(2))
    return f"{bank}{num}"

nozzle_to_idx = {canonical_nozzle(n): i for i, n in enumerate(nozzles)}

# ---- helpers for pressure extraction ----
pressure_re = re.compile(r'(-?\d+(?:\.\d+)?)\s*mbar', re.IGNORECASE)

def extract_pressure_from_text(text) -> float:
    text = "" if text is None else str(text)
    m = pressure_re.search(text)
    if m:
        return float(m.group(1))
    nums = re.findall(r'-?\d+(?:\.\d+)?', text)
    return float(nums[-1]) if nums else np.nan

def find_pressure_in_df(df, file_colname="file") -> float:
    if file_colname not in df.columns:
        file_colname = df.columns[0]
    for val in df[file_colname]:
        p = extract_pressure_from_text(val)
        if np.isfinite(p):
            return p
    return np.nan

def get_replicate_from_filename(fname: str):
    m = re.search(r'_(\d+)_labels?', fname.lower())
    return int(m.group(1)) if m else None

# ---- allocate full matrix ----
labels_all  = np.full((n_nozzles, n_pressures), "", dtype=object)
filled_mask = np.zeros((n_nozzles, n_pressures), dtype=bool)

# ---- collect label files ----
label_files = sorted(
    glob.glob(os.path.join(LABELS_DIR, "*_Labels.csv")) +
    glob.glob(os.path.join(LABELS_DIR, "*_Labels.xlsx"))
)
print(f"Found {len(label_files)} label files total in {LABELS_DIR}")

# ---- main loop ----
used_files = 0
skipped_outside = 0
skipped_tolerance = 0

for path in label_files:
    fname = os.path.basename(path)
    rep = get_replicate_from_filename(fname)
    if rep != replicate:
        continue

    df = pd.read_csv(path) if path.lower().endswith(".csv") else pd.read_excel(path)
    if df.empty:
        continue

    p_val = find_pressure_in_df(df, file_colname="file")
    if not np.isfinite(p_val):
        continue

    # Filter using dataset-derived range (not hard-coded!)
    if not (P_LOW - TOL <= p_val <= P_HIGH + TOL):
        skipped_outside += 1
        continue

    j = int(np.argmin(np.abs(pressures - p_val)))
    diff = float(abs(pressures[j] - p_val))

    # Require reasonable match to grid (prevents mapping the wrong pressure)
    if diff > TOL:
        skipped_tolerance += 1
        continue

    # columns
    label_col    = "Label"      if "Label" in df.columns else df.columns[3]
    nozzleid_col = "Nozzle_id"  if "Nozzle_id" in df.columns else df.columns[5]

    for _, r in df.iterrows():
        label = str(r[label_col]).strip()
        if not label:
            continue

        key = canonical_nozzle(str(r[nozzleid_col]).strip())
        if key not in nozzle_to_idx:
            continue

        i = nozzle_to_idx[key]
        labels_all[i, j] = label
        filled_mask[i, j] = True

    used_files += 1

# ---- sanity / coverage ----
n_filled = int(filled_mask.sum())
print(f"\nUsed {used_files} label files for replicate {replicate}.")
print(f"Skipped outside range: {skipped_outside} | skipped by tolerance: {skipped_tolerance}")
print(f"Filled {n_filled} / {n_nozzles * n_pressures} nozzle×pressure slots")

# Compact per-pressure fill (only show pressures that have ANY labels)
per_p = filled_mask.sum(axis=0)
nonzero = np.where(per_p > 0)[0]
print("\nFilled counts (only non-zero pressure steps):")
for j in nonzero:
    print(f"  p={pressures[j]:.1f} (idx {j}): {int(per_p[j])}/{n_nozzles}")

labels_npz_path = os.path.join(fit_dir, f"labels_rep_{replicate}_{version}.npz")
np.savez_compressed(
    labels_npz_path,
    labels=labels_all,
    pressures=pressures,
    nozzles=np.array(nozzles, dtype=object),
)
print(f"\n Saved labels NPZ to:\n  {labels_npz_path}")


n_nozzles: 1280 n_pressures: 95
Pressure grid range: [-87.600, -50.000] mBar | step≈0.400 | tol=0.250
LABELS_DIR: /content/drive/MyDrive/data/labels/2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps_ISPM_PH4
Found 0 label files total in /content/drive/MyDrive/data/labels/2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps_ISPM_PH4

Used 0 label files for replicate 2.
Skipped outside range: 0 | skipped by tolerance: 0
Filled 0 / 121600 nozzle×pressure slots

Filled counts (only non-zero pressure steps):

 Saved labels NPZ to:
  /content/drive/MyDrive/data/fit/labels_rep_2_lowering.npz


**Merge labels and parameters**

In [ ]:
import os
import numpy as np

# --- CONFIG (same as before) ---
replicate = 2
version   = "lowering"
fit_dir   = "/content/drive/MyDrive/data/fit"

# path to labels NPZ we just created
labels_npz_path = os.path.join(
    fit_dir,
    f"labels_rep_{replicate}_{version}.npz"
)

# existing merged (bank-corrected) fit file
merged_path = os.path.join(
    fit_dir,
    f"merged_rep_{replicate}_{version}_bankcorrected_params6_Q_rmse_nrmse.npz"
)

print("Loading:")
print("  merged:", merged_path)
print("  labels:", labels_npz_path)

merged   = np.load(merged_path, allow_pickle=True)
labels_n = np.load(labels_npz_path, allow_pickle=True)

labels_all = labels_n["labels"]
press_l    = labels_n["pressures"]
nozz_l     = labels_n["nozzles"]

press_m    = merged["pressures"]
nozz_m     = merged["nozzles"]

print("labels_all.shape:", labels_all.shape)
print("params6_all.shape:", merged["params6_all"].shape)

# --- consistency checks: same grid & ordering ---
assert np.allclose(press_l, press_m), " pressure grids in labels vs merged do not match"
assert np.array_equal(nozz_l, nozz_m), " nozzle ordering in labels vs merged do not match"

n_nozzles  = nozz_m.shape[0]
n_pressure = press_m.shape[0]

# ensure labels are (nozzle, pressure); transpose if needed
if labels_all.shape[0] == n_nozzles and labels_all.shape[1] == n_pressure:
    print("Labels already in shape (nozzle, pressure).")
elif labels_all.shape[0] == n_pressure and labels_all.shape[1] == n_nozzles:
    print("Transposing labels from (pressure, nozzle) -> (nozzle, pressure).")
    labels_all = labels_all.T
else:
    raise ValueError(
        f"Unexpected labels shape {labels_all.shape} "
        f"for {n_nozzles} nozzles × {n_pressure} pressures."
    )

# final consistency check with fit array
assert labels_all.shape[:2] == merged["params6_all"].shape[:2], \
    " labels shape does not match params6_all leading dimensions"

# --- save merged+labels NPZ ---
merged_with_labels_path = os.path.join(
    fit_dir,
    f"merged_rep_{replicate}_{version}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
)

np.savez_compressed(
    merged_with_labels_path,
    params6_all   = merged["params6_all"],
    Q_all         = merged["Q_all"],
    rmse_all      = merged["rmse_all"],
    nrmse_all     = merged["nrmse_all"],
    pressures     = press_m,
    nozzles       = nozz_m,
    inverted_mask = merged.get("inverted_mask", None),
    labels        = labels_all,
)

print("Saved merged+labels NPZ to:")
print(" ", merged_with_labels_path)

Loading:
  merged: /content/drive/MyDrive/data/fit/merged_rep_2_lowering_bankcorrected_params6_Q_rmse_nrmse.npz
  labels: /content/drive/MyDrive/data/fit/labels_rep_2_lowering.npz
labels_all.shape: (1280, 95)
params6_all.shape: (1280, 95, 6)
Labels already in shape (nozzle, pressure).
Saved merged+labels NPZ to:
  /content/drive/MyDrive/data/fit/merged_rep_2_lowering_bankcorrected_params6_Q_rmse_nrmse_labels.npz


**Interactive viewer**

In [ ]:
# ==== Interactive viewer: (all) -> browse by nozzle/pressure
#      label!=all -> browse by "Label idx" slider (k-th occurrence of that label) ====

import os, json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display

# If you're on Colab, this helps widgets render properly
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass


# -------------------------
# CONFIG
# -------------------------
base_pre  = "/content/drive/MyDrive/data/preprocessed"
base_ring = "/content/drive/MyDrive/data/ringdown"
fit_dir   = "/content/drive/MyDrive/data/fit"

DATASETS = ["1_lowering", "2_lowering", "1_increasing"]
GOOD_LABEL = "Jetting"
INVERT_BANKS = ("A", "C")

fs = 4.125e6
dt = 1.0 / fs

def damped_sine_baseline_tau(t, A, tau, f, phi, C, D):
    tau = np.clip(tau, 1e-12, None)
    return C + D*t + A*np.exp(-t/tau)*np.sin(2*np.pi*f*t + phi)

def reflect_about_mid(y):
    y_mid = 0.5*(y.min() + y.max())
    return 2.0*y_mid - y

def compute_inverted_mask_from_nozzles(nozzles, n_pressures, invert_banks=("A","C")):
    nozz_arr = np.array([str(n) for n in nozzles], dtype=object)
    banks    = np.array([s[0].upper() if len(s) else "?" for s in nozz_arr], dtype=object)
    mask_noz = np.isin(banks, list(invert_banks))
    return np.repeat(mask_noz[:, None], n_pressures, axis=1)

def bank_from_index(nozzles, i):
    s = str(nozzles[i])
    return s[0].upper() if s else "?"

def is_empty_label(x):
    return str(x).strip() in ("", "nan", "None", "NaN")

# -------------------------
# STATE
# -------------------------
STATE = {}

def load_dataset(dataset_key: str):
    rep = int(dataset_key.split("_")[0])
    ver = dataset_key.split("_", 1)[1]

    tensor_path = os.path.join(base_ring, f"tensor_replicate_{rep}_{ver}_ringdown.npy")
    press_path  = os.path.join(base_pre,  f"pressures_replicate_{rep}_{ver}.json")
    nozz_path   = os.path.join(base_pre,  f"nozzles_replicate_{rep}_{ver}.json")

    tensor = np.load(tensor_path, allow_pickle=True)
    with open(press_path, "r") as f:
        pressures = np.array(json.load(f), dtype=float)
    with open(nozz_path, "r") as f:
        nozzles = np.array(json.load(f), dtype=object)

    n_nozzles, n_samples, n_pressures = tensor.shape
    t = np.arange(n_samples) * dt

    merged_labels_path = os.path.join(
        fit_dir,
        f"merged_rep_{rep}_{ver}_bankcorrected_params6_Q_rmse_nrmse_labels.npz"
    )
    if not os.path.exists(merged_labels_path):
        raise FileNotFoundError(f"Missing merged+labels NPZ:\n  {merged_labels_path}")

    z = np.load(merged_labels_path, allow_pickle=True)

    params6_all = z["params6_all"]
    Q_all       = z.get("Q_all", None)

    pressures_m = z["pressures"].astype(float)
    nozzles_m   = z["nozzles"]
    labels_all  = z["labels"]

    if not np.allclose(pressures, pressures_m):
        raise ValueError(" pressures mismatch between raw tensor and merged+labels NPZ")
    if not np.array_equal(nozzles, nozzles_m):
        raise ValueError(" nozzles mismatch between raw tensor and merged+labels NPZ")

    if labels_all.shape == (n_nozzles, n_pressures):
        pass
    elif labels_all.shape == (n_pressures, n_nozzles):
        labels_all = labels_all.T
    else:
        raise ValueError(f"Unexpected labels shape {labels_all.shape} for {n_nozzles}×{n_pressures}")

    inv = z.get("inverted_mask", None)
    if inv is None or getattr(inv, "shape", None) != (n_nozzles, n_pressures) or int(np.sum(inv)) == 0:
        inv = compute_inverted_mask_from_nozzles(nozzles, n_pressures, invert_banks=INVERT_BANKS)

    label_to_indices = defaultdict(list)
    outlier_indices  = []
    for i in range(n_nozzles):
        for j in range(n_pressures):
            lbl = str(labels_all[i, j]).strip()
            if is_empty_label(lbl):
                continue
            label_to_indices[lbl].append((i, j))
            if lbl != GOOD_LABEL:
                outlier_indices.append((i, j))

    label_list = sorted(label_to_indices.keys())

    # quick counts
    counts = {lbl: len(v) for lbl, v in label_to_indices.items()}
    ordered = []
    if GOOD_LABEL in counts:
        ordered.append(GOOD_LABEL)
    ordered += [k for k in sorted(counts.keys()) if k != GOOD_LABEL]
    print(f"[{dataset_key}] " + " | ".join([f"{k}={counts[k]}" for k in ordered]) + f" | Outlier={len(outlier_indices)}")

    STATE.clear()
    STATE.update(dict(
        dataset_key=dataset_key,
        tensor=tensor,
        pressures=pressures,
        nozzles=nozzles,
        t=t,
        params6_all=params6_all,
        Q_all=Q_all,
        labels_all=labels_all,
        inverted_mask=inv,
        label_to_indices=label_to_indices,
        outlier_indices=outlier_indices,
        n_nozzles=n_nozzles,
        n_pressures=n_pressures,
    ))

    return n_nozzles, n_pressures, label_list


def view_signal(nozzle_idx, pressure_idx):
    y_raw = STATE["tensor"][nozzle_idx, :, pressure_idx]
    if STATE["inverted_mask"][nozzle_idx, pressure_idx]:
        return reflect_about_mid(y_raw)
    return y_raw


# -------------------------
# WIDGETS
# -------------------------
dataset_dd = widgets.Dropdown(options=DATASETS, value="2_lowering", description="Dataset")

nozzle_slider = widgets.IntSlider(value=0, min=0, max=1, step=1,
                                  description="Nozzle", continuous_update=False)
pressure_slider = widgets.IntSlider(value=0, min=0, max=1, step=1,
                                    description="Pressure idx", continuous_update=False)

label_filter = widgets.Dropdown(options=["(all)"], value="(all)", description="Label filter")

# <<< THIS is the slider you asked for >>>
label_idx_slider = widgets.IntSlider(
    value=0, min=0, max=0, step=1,
    description="Label idx",
    continuous_update=False,
    disabled=True
)

auto_ylim  = widgets.Checkbox(value=True, description="Auto y-limits")
show_resid = widgets.Checkbox(value=False, description="Show residual")

status = widgets.HTML(value="")

def set_status(msg):
    status.value = f"<div style='font-family: monospace; padding: 6px 0;'>{msg}</div>"

def current_pairs_for_label(lf: str):
    if lf == "Outlier":
        return STATE["outlier_indices"]
    return STATE["label_to_indices"].get(lf, [])

def on_dataset_change(change):
    n_nozzles, n_pressures, label_list = load_dataset(change["new"])

    nozzle_slider.max   = n_nozzles - 1
    pressure_slider.max = n_pressures - 1
    nozzle_slider.value = 0
    pressure_slider.value = 0

    label_filter.options = ["(all)", "Outlier"] + label_list
    label_filter.value   = "(all)"

    # enforce mode update
    on_label_change({"new": "(all)"})

def on_label_change(change):
    lf = change["new"]

    if lf == "(all)":
        label_idx_slider.disabled = True
        label_idx_slider.max = 0
        label_idx_slider.value = 0

        nozzle_slider.disabled = False
        pressure_slider.disabled = False

        set_status("Mode: (all) → browse with Nozzle + Pressure idx")
        return

    # label mode
    pairs = current_pairs_for_label(lf)
    n = len(pairs)

    nozzle_slider.disabled = True
    pressure_slider.disabled = True

    if n == 0:
        label_idx_slider.disabled = True
        label_idx_slider.max = 0
        label_idx_slider.value = 0
        set_status(f"Mode: {lf} → 0 matches")
    else:
        label_idx_slider.disabled = False
        label_idx_slider.max = n - 1
        label_idx_slider.value = 0
        set_status(f"Mode: {lf} → {n} matches (use Label idx slider)")

dataset_dd.observe(on_dataset_change, names="value")
label_filter.observe(on_label_change, names="value")


# -------------------------
# PLOT
# -------------------------
def plot_fit(nozzle_idx, pressure_idx, lf, label_idx, auto_y, show_resid):
    if not STATE:
        return

    # If label mode, override (nozzle, pressure) using label_idx_slider
    if lf != "(all)":
        pairs = current_pairs_for_label(lf)
        if len(pairs) == 0:
            plt.figure(figsize=(9, 3))
            plt.axis("off")
            plt.text(0.01, 0.5, f"No matches for label '{lf}'", fontsize=12)
            plt.show()
            return
        label_idx = int(np.clip(label_idx, 0, len(pairs) - 1))
        nozzle_idx, pressure_idx = pairs[label_idx]

    t         = STATE["t"]
    pressures = STATE["pressures"]
    nozzles   = STATE["nozzles"]

    y = view_signal(nozzle_idx, pressure_idx)

    A, tau, f, phi, C, D = STATE["params6_all"][nozzle_idx, pressure_idx, :]
    yfit = damped_sine_baseline_tau(t, A, tau, f, phi, C, D)

    rmse  = float(np.sqrt(np.mean((y - yfit)**2)))
    nrmse = float(rmse / (np.ptp(y) + 1e-12))

    Q_all = STATE["Q_all"]
    if Q_all is not None and np.isfinite(Q_all[nozzle_idx, pressure_idx]):
        Q_val = float(Q_all[nozzle_idx, pressure_idx])
    else:
        Q_val = float(np.pi * f * tau) if (np.isfinite(f) and np.isfinite(tau) and tau > 0) else np.nan

    inverted_flag = bool(STATE["inverted_mask"][nozzle_idx, pressure_idx])
    lbl = str(STATE["labels_all"][nozzle_idx, pressure_idx]).strip()

    bank = bank_from_index(nozzles, nozzle_idx)
    pval = float(pressures[pressure_idx])
    nozzle_name = str(nozzles[nozzle_idx])

    if show_resid:
        import matplotlib.gridspec as gridspec
        fig = plt.figure(figsize=(10, 6))
        gs  = gridspec.GridSpec(2, 1, height_ratios=[3, 1])
        ax1 = fig.add_subplot(gs[0])
        ax2 = fig.add_subplot(gs[1], sharex=ax1)
        ax = ax1
    else:
        fig, ax = plt.subplots(figsize=(10, 5))

    ax.plot(t*1e6, y, lw=2, label="Measured")
    ax.plot(t*1e6, yfit, "--", lw=2, label="Fit")

    if lf == "(all)":
        mode_line = "Mode: (all)"
    else:
        n = len(current_pairs_for_label(lf))
        mode_line = f"Mode: {lf} | label idx {int(label_idx)+1}/{n}"

    ax.set_title(
        f"{STATE['dataset_key']} — {mode_line}\n"
        f"Bank {bank}, Nozzle #{nozzle_idx} ({nozzle_name}) @ {pval:.1f} mbar | Label='{lbl}' | "
        f"NRMSE={nrmse:.3f} | Q≈{Q_val:.2f} | Inverted={inverted_flag}"
    )
    ax.set_xlabel("Time (µs)")
    ax.set_ylabel("Amplitude (a.u.)")
    ax.grid(True)
    ax.legend()

    if not auto_y:
        m = max(np.max(np.abs(y)), np.max(np.abs(yfit)))
        ax.set_ylim(-m*1.05, m*1.05)

    if show_resid:
        resid = y - yfit
        ax2.plot(t*1e6, resid, lw=1.5)
        ax2.axhline(0, ls="--", lw=1)
        ax2.set_ylabel("Residual")
        ax2.set_xlabel("Time (µs)")
        ax2.grid(True)
        if not auto_y:
            mr = np.max(np.abs(resid))
            ax2.set_ylim(-mr*1.05, mr*1.05)

    plt.tight_layout()
    plt.show()


# -------------------------
# RUN
# -------------------------
on_dataset_change({"new": dataset_dd.value})

out = widgets.interactive_output(
    plot_fit,
    dict(
        nozzle_idx=nozzle_slider,
        pressure_idx=pressure_slider,
        lf=label_filter,
        label_idx=label_idx_slider,   # <<< THIS drives label browsing
        auto_y=auto_ylim,
        show_resid=show_resid,
    )
)

# Show controls (label slider is ALWAYS displayed; just disabled in (all) mode)
display(dataset_dd)
display(HBox([nozzle_slider, pressure_slider]))
display(label_filter)
display(label_idx_slider)
display(HBox([auto_ylim, show_resid]))
display(status)
display(out)

[2_lowering] Jetting=116837 | Deviated=1173 | Intermittent=447 | Non-jetting=3139 | Other=4 | Outlier=4763


Dropdown(description='Dataset', index=1, options=('1_lowering', '2_lowering', '1_increasing'), value='2_loweri…

Dropdown(description='Label filter', options=('(all)', 'Outlier', 'Deviated', 'Intermittent', 'Jetting', 'Non-…

IntSlider(value=0, continuous_update=False, description='Label idx', disabled=True, max=0)

HTML(value="<div style='font-family: monospace; padding: 6px 0;'>Mode: (all) → browse with Nozzle + Pressure i…

Output()

[1_increasing] Jetting=12506 | Deviated=353 | Intermittent=13 | Non-jetting=1173 | Other=35 | Outlier=1574
[1_lowering] Jetting=117375 | Deviated=777 | Intermittent=507 | Non-jetting=2938 | Other=3 | Outlier=4225
